# **LAB 1**

This lab focuses on implementing and testing shifted/rotated benchmark functions commonly used in optimization research. The tasks include:

General Function Wrapper: Create a Function class that:

* Evaluates solutions while enforcing bounds (x_lower, x_upper).

* Shifts the input using a predefined optimum vector (o) and adds a bias (f_bias).

Benchmark Functions: Implement five CEC-inspired functions:

* F1: Shifted Sphere Function (∑z²).

* F2: Shifted Schwefel’s Problem 1.2 (∑(cumsum(z))²).

* F3: Shifted Rotated High Conditioned Elliptic Function (uses a rotation matrix M).

* F4: Shifted Schwefel’s Problem 1.2 with noise.

* F5: Schwefel’s Problem 2.6 with global optimum on bounds.

Validation: Test each function with randomized inputs to ensure correct shifting, clipping, and bias handling.

In [45]:
import numpy as np
np.random.seed(42)

In [46]:
class Function:
    def __init__(self, function, dimension, x_lower= -np.inf, x_upper=np.inf, o=None, f_bias=0):
        self.function = function
        self.dimension = dimension
        self.x_lower = np.array(x_lower)
        self.x_upper = np.array(x_upper)
        self.o = o if o is not None else np.random.uniform(x_lower, x_upper, dimension)
        self.f_bias = f_bias

    def __call__(self, x):
        return self.evaluate(x)

    def evaluate(self, x):
        x = np.array(x)
        if x.shape != (self.dimension,):
            raise ValueError(f"Input x must have shape ({self.dimension},)")
        x = np.clip(x, self.x_lower, self.x_upper)
        z = x - self.o
        return self.function(z) + self.f_bias

In [47]:
dimension = 10
x_lower = [-100] * dimension
x_upper = [100] * dimension

x_test = np.random.uniform(-100, 100, dimension)
o = np.random.uniform(-80, 80, dimension)
bias = -450

## Examples

In [48]:
# F1 : Shifted Sphere Function
def F1(z):
    return np.sum(z**2)

# F2: Shifted Schwefel's Problem 1.2
def F2(z):
    return np.sum(np.cumsum(z)**2)

# F3: Shifted Rotated High Conditioned Elliptic Function
def F3(z, M):
    z = np.dot(M, z)
    return np.sum((10**(6 * np.arange(len(z)) / (len(z) - 1))) * z**2)

# F4: Shifted Schwefel's Problem 1.2 with Noise in Fitness
def F4(z):
    base = np.sum(np.cumsum(z)**2)
    noise = 1 + 0.4 * abs(np.random.normal(0, 1))
    return base * noise

# F5: Schwefel's Problem 2.6 with Global Optimum on Bounds
def F5(z, A, B):
    return np.max(np.abs(A @ z - B))

In [49]:
f1 = Function(F1, dimension, x_lower, x_upper, o, f_bias=bias)
print("F1 result:", f1(x_test))


F1 result: 22107.823921702915


In [50]:
f2 = Function(F2, dimension, x_lower, x_upper, o, f_bias=bias)
print("F2 result:", f2(x_test))



F2 result: 117819.13029652895


In [51]:
M = np.random.randn(dimension, dimension)
f3 = Function(lambda z: F3(z, M), dimension, x_lower, x_upper, o, f_bias=bias)
print("F3 result:", f3(x_test))


F3 result: 98536552527.04808


In [52]:
f4 = Function(F4, dimension, x_lower, x_upper, o, f_bias=bias)
print("F4 result:", f4(x_test))


F4 result: 119461.26262216597


In [53]:
A = np.random.randint(-500, 501, size=(dimension, dimension))
B = A @ o
f5 = Function(lambda z: F5(z, A, B), dimension, x_lower, x_upper, o, f_bias=bias)
print("F5 result:", f5(x_test))

F5 result: 170507.64017914695


# **LAB 2**

This lab implements a self-adaptive population-based random search algorithm. Key components include:

Self-Adaptive Mutation: Adjusts the step size (alpha) dynamically based on fitness improvements, scaled by alpha_change_rate.

Population Management: Generates multiple agents per iteration, replacing parent agents only if offspring perform better.

Variable Population Size: Maintains a flexible population size to balance exploration and exploitation.

Bounded Search: Ensures solutions stay within predefined bounds (x_lower, x_upper).
The algorithm iterates through mutation, evaluation, and replacement phases, emphasizing adaptive exploration in high-dimensional spaces.

In [54]:
class PopulationV3_SelfAdaptive:
    def __init__(self, func:Function, population_size, alpha, alpha_change_rate, max_iterations):
        self.func = func
        self.population_size = population_size
        self.alpha = alpha
        self.alpha_change_rate = alpha_change_rate
        self.max_iterations = max_iterations
        self.population = self.initialize_population()

    def initialize_population(self):
        return [np.random.uniform(self.func.x_lower, self.func.x_upper, self.func.dimension)
                for _ in range(self.population_size)]

    def evaluate_population(self, population):
        return [self.func(agent) for agent in population]

    def generate_new_agents(self, agent):
        new_agents = []
        for _ in range(self.population_size):
            rand_vector = np.random.uniform(-1, 1, self.func.dimension)
            alpha = self.alpha + self.alpha * np.random.uniform(-1, 1)
            new_agent = agent + alpha * rand_vector
            new_agent = np.clip(new_agent, self.func.x_lower, self.func.x_upper)
            new_agents.append(new_agent)
        return new_agents

    def evolve(self):
        for iteration in range(self.max_iterations):
            population_fitness = self.evaluate_population(self.population)
            new_population = []

            for agent in self.population:
                new_agents = self.generate_new_agents(agent)
                new_agents_fitness = self.evaluate_population(new_agents)

                better_agents = [new_agents[i] for i in range(len(new_agents))
                                 if new_agents_fitness[i] < self.func(agent)]
                if better_agents:
                    new_population.extend(better_agents)
                else:
                    new_population.append(agent)

            self.population = new_population

            self.alpha += self.alpha * self.alpha_change_rate

            print(f"Iteration {iteration + 1}/{self.max_iterations}, Best fitness: {min(population_fitness)}")

        return min(self.evaluate_population(self.population))

In [55]:
# Exa
def sphere_function(x):
    return np.sum(x**2)

# Inițializarea funcției și parametrii algoritmului
func = Function(sphere_function, dimension=10, x_lower=-5, x_upper=5)
algorithm = PopulationV3_SelfAdaptive(func, population_size=2, alpha=0.5, alpha_change_rate=0.01, max_iterations=10)

# Rularea algoritmului
best_fitness = algorithm.evolve()
print(f"Best fitness found: {best_fitness}")


Iteration 1/10, Best fitness: 150.56417368898212
Iteration 2/10, Best fitness: 147.21147795397664
Iteration 3/10, Best fitness: 146.42583992621582
Iteration 4/10, Best fitness: 146.42583992621582
Iteration 5/10, Best fitness: 146.0382936769243
Iteration 6/10, Best fitness: 134.11971419191977
Iteration 7/10, Best fitness: 131.29650308715668
Iteration 8/10, Best fitness: 102.0888214427925
Iteration 9/10, Best fitness: 98.88906196155037
Iteration 10/10, Best fitness: 95.25361800098383
Best fitness found: 94.55379831645315


# **Lab 3**

This lab focuses on a Canonical Genetic Algorithm (CGA) with a greedy replacement strategy:

Binary Representation: Encodes real-valued solutions into 64-bit binary strings for crossover/mutation.

Roulette Wheel Selection: Selects parents probabilistically based on fitness.

Single-Point Crossover & Bit-Flip Mutation: Standard genetic operators with fixed probabilities (pc, pm).

Greedy Replacement: Replaces the worst offspring in each generation with the best parent to preserve elitism.
The implementation includes utilities for binary-to-float conversion and fitness-driven evolution, targeting discrete optimization problems.

In [56]:
def bin2float(binary_list):
    """Convert 64-bit binary string to float"""
    rez = ''.join(binary_list)
    h = int(rez, 2).to_bytes(8, byteorder="big")
    return struct.unpack('>d', h)[0]

def float2bin(float_val):
    """Convert float to 64-bit binary representation"""
    [d] = struct.unpack(">Q", struct.pack(">d", float_val))
    return list(f'{d:064b}')

def solution2binary(solution_x):
    """Convert real-valued solution to binary string"""
    binary_sol = []
    for val in solution_x:
        a = float2bin(val)
        binary_sol += a
    return binary_sol

def binarysolution2float(binary_sol):
    """Convert binary string back to real-valued solution"""
    n = 64
    solution_x = []
    bitss = [binary_sol[i:i + n] for i in range(0, len(binary_sol), n)]
    for bits in bitss:
        solution_x.append(bin2float(bits))
    return solution_x

In [57]:
class CGA_greedy:
    def __init__(self, function, dimension, population_size, pc, pm, max_nfe):
        self.function = function
        self.dimension = dimension
        self.population_size = population_size
        self.pc = pc
        self.pm = pm
        self.max_nfe = max_nfe
        self.population = self.initialize_population()
        self.best_individual = None
        self.best_fitness = float('inf')
        self.nfe = 0  # Number of Function Evaluations

    def initialize_population(self):
        """Initialize population with real-valued solutions converted to binary"""
        population_real = np.random.uniform(
            self.function.x_lower,
            self.function.x_upper,
            (self.population_size, self.dimension)
        )
        return [solution2binary(individual) for individual in population_real]

    def evaluate_fitness(self, individual):
        """Evaluate fitness of a binary-encoded individual"""
        decoded = binarysolution2float(individual)
        fitness = self.function(decoded)
        self.nfe += 1
        return fitness

    def selection(self):
        """Roulette wheel selection based on fitness"""
        fitnesses = np.array([self.evaluate_fitness(ind) for ind in self.population])
        total_fitness = np.sum(fitnesses)
        probabilities = fitnesses / total_fitness
        selected_indices = np.random.choice(
            np.arange(self.population_size),
            size=2,
            p=probabilities,
            replace=False
        )
        return [self.population[i] for i in selected_indices]

    def crossover(self, parent1, parent2):
        """Single-point crossover"""
        if np.random.rand() < self.pc:
            crossover_point = np.random.randint(1, len(parent1))
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
            return child1, child2
        return parent1.copy(), parent2.copy()

    def mutation(self, individual):
        """Bit-flip mutation"""
        for j in range(len(individual)):
            if np.random.rand() < self.pm:
                individual[j] = '0' if individual[j] == '1' else '1'
        return individual

    def replace_worst_with_best(self, children, parents):
        """Greedy replacement strategy"""
        children_fitness = np.array([self.evaluate_fitness(child) for child in children])
        parents_fitness = np.array([self.evaluate_fitness(parent) for parent in parents])
        worst_child_index = np.argmax(children_fitness)
        best_parent_index = np.argmin(parents_fitness)
        children[worst_child_index] = parents[best_parent_index].copy()
        return children

    def evolve(self):
        """Main evolution loop"""
        while self.nfe < self.max_nfe:
            new_population = []
            for _ in range(self.population_size // 2):
                # Selection and Crossover
                parents = self.selection()
                child1, child2 = self.crossover(parents[0], parents[1])

                # Mutation
                child1 = self.mutation(child1)
                child2 = self.mutation(child2)

                # Greedy Replacement
                children = self.replace_worst_with_best([child1, child2], parents)
                new_population.extend(children)

            self.population = np.array(new_population)

            # Update best individual
            current_best = self.population[np.argmin([self.evaluate_fitness(ind) for ind in self.population])]
            current_best_fitness = self.evaluate_fitness(current_best)

            if current_best_fitness < self.best_fitness:
                self.best_fitness = current_best_fitness
                self.best_individual = current_best.copy()

            print(f"NFE: {self.nfe}, Best Fitness: {self.best_fitness}")

        return binarysolution2float(self.best_individual), self.best_fitness

In [58]:
if __name__ == "__main__":
    # Test with Sphere function (requires Function class from Lab 1)
    def sphere_function(x):
        return np.sum(x**2)

    # Algorithm parameters
    population_size = 50
    pc = 0.8
    pm = 0.1
    max_nfe = 1000
    dimension = 10

    # Initialize function and algorithm
    func = Function(sphere_function, dimension, x_lower=-5.12, x_upper=5.12)
    cga = CGA_greedy(func, dimension, population_size, pc, pm, max_nfe)

    # Run evolution
    best_individual, best_fitness = cga.evolve()

    print(f"Best solution: {best_individual}")
    print(f"Best fitness: {best_fitness}")

NFE: 1401, Best Fitness: 41.27142791497669
Best solution: [0.19539747979487299, -0.2131775730715546, -4.857425246142099, -1.625622245108159, -1.2267968636328237, -1.036054723615739, 0.8209650608878825, 0.3440900784832355, 1.1049481502048728, 2.7124045981913936]
Best fitness: 41.27142791497669


C:\Users\Tempest\AppData\Local\Temp\ipykernel_792\2652646617.py:18: RuntimeWarning: invalid value encountered in subtract
  z = x - self.o


# **Lab 4**

This lab implements a Real-Coded Genetic Algorithm (RGA) with adaptive parameters:

Real-Valued Representation: Directly manipulates real-number solutions without encoding.

BLX-0.1 Crossover: Generates offspring within expanded bounds around parent solutions to enhance diversity.

Gaussian Mutation: Perturbs solutions using a normal distribution for fine-grained exploration.

Adaptive Parameters: Dynamically adjusts crossover (pc) and mutation (pm) probabilities based on fitness feedback.
The algorithm balances exploration and exploitation through adaptive operators, tailored for continuous optimization.

In [59]:
import numpy as np

class RGA_1_adaptive:
    def __init__(self, function: Function, dimension, population_size, pc_initial, pm_initial, max_nfe):
        self.function = function
        self.dimension = dimension
        self.population_size = population_size
        self.pc_initial = pc_initial
        self.pm_initial = pm_initial
        self.max_nfe = max_nfe
        self.population = self.initialize_population()
        self.best_individual = None
        self.best_fitness = float('inf')
        self.nfe = 0

    def initialize_population(self):
        return np.random.uniform(self.function.x_lower, self.function.x_upper,
                               (self.population_size, self.function.dimension))

    def evaluate_fitness(self, individual):
        fitness = self.function(individual)
        self.nfe += 1
        return fitness

    def linear_crossover(self, parent1, parent2):
        alpha = np.random.rand()
        # Ensure the shape of the offspring remains the same as the parents
        child1 = alpha * parent1 + (1 - alpha) * parent2
        child2 = (1 - alpha) * parent1 + alpha * parent2
        return child1, child2

    def non_uniform_mutation(self, individual, t, t_max, b=5):
        tau = np.random.choice([-1, 1], size=self.dimension) # Generate tau for each dimension
        r = np.random.rand(self.dimension) # Generate r for each dimension
        mutation_factor = (1 - r**(1 - t / t_max)**b)
        # Ensure the shape of the mutated individual remains the same
        mutated_individual = individual + tau * mutation_factor * (self.function.x_upper - self.function.x_lower)
        return mutated_individual

    def roulette_wheel_selection(self, fitnesses):
        total_fitness = np.sum(fitnesses)
        probabilities = fitnesses / total_fitness
        selected_indices = np.random.choice(np.arange(self.population_size), size=2, p=probabilities, replace=False)
        return self.population[selected_indices]

    def adapt_pc_pm(self, t, t_max):
        self.pc = self.pc_initial * (1 - t / t_max)
        self.pm = self.pm_initial * (t / t_max)

    def evolve(self):
        t_max = self.max_nfe // self.population_size  # Numărul maxim de generații
        while self.nfe < self.max_nfe:
            t = self.nfe // self.population_size  # Generația curentă

            # Adaptarea pc și pm
            self.adapt_pc_pm(t, t_max)

            new_population = []
            for _ in range(self.population_size // 2):
                # Selecția părinților
                parents = self.roulette_wheel_selection([self.evaluate_fitness(ind) for ind in self.population])

                # Încrucișare
                if np.random.rand() < self.pc:
                    # Pass individual parents to linear_crossover
                    child1, child2 = self.linear_crossover(parents[0], parents[1])
                else:
                    child1, child2 = parents[0].copy(), parents[1].copy()

                # Mutație
                child1 = self.non_uniform_mutation(child1, t, t_max)
                child2 = self.non_uniform_mutation(child2, t, t_max)

                new_population.extend([child1, child2])

            self.population = np.array(new_population)

            # Actualizare cel mai bun individ
            current_best = self.population[np.argmin([self.evaluate_fitness(ind) for ind in self.population])]
            current_best_fitness = self.evaluate_fitness(current_best)
            if current_best_fitness < self.best_fitness:
                self.best_fitness = current_best_fitness
                self.best_individual = current_best.copy()

        return self.best_individual, self.best_fitness

In [60]:
# Definirea funcției de test (exemplu - funcția Sphere)
def sphere_function(x):
    return np.sum(x**2)

# Parametrii algoritmului
population_size = 50
pc_initial = 0.8
pm_initial = 0.1
max_nfe = 1000

# Dimensiunea problemei
dimension = 10

# Crearea funcției de test
function = Function(sphere_function, dimension, x_lower=-5.12, x_upper=5.12)

# Crearea algoritmului RGA_1_adaptive
rga = RGA_1_adaptive(function, dimension, population_size, pc_initial, pm_initial, max_nfe)

# Executarea algoritmului
best_individual, best_fitness = rga.evolve()

# Afișarea rezultatelor
print(f"Cel mai bun individ: {best_individual}")
print(f"Fitness-ul cel mai bun: {best_fitness}")

Cel mai bun individ: [ -1.06986847 -11.6849258   -1.70182871  -7.1177488   -5.25270925
   4.7620541   -7.44356726   2.47827057   6.59983964   0.67015133]
Fitness-ul cel mai bun: 79.63615855571462


# **Lab 5**

This lab implements a Differential Evolution (DE) variant (DE/best/2/exp):

Best/2 Mutation: Creates donor vectors using the best solution and two differential terms for strong exploitation.

Exponential Crossover: Inherits contiguous parameter blocks from the donor vector to maintain solution coherence.

Fixed Control Parameters: Uses constant scaling factor (F=0.8) and crossover rate (CR=0.9).

Greedy Selection: Retains better solutions between trial and target vectors to drive convergence.
The algorithm excels in numerical optimization, particularly for high-dimensional continuous landscapes.

In [61]:
import numpy as np

class DE_best_1_exp:
    def __init__(self, function: Function, population_size, F, CR, max_nfe):
        self.function = function
        self.population_size = population_size
        self.F = F
        self.CR = CR
        self.max_nfe = max_nfe
        self.population = self.initialize_population()
        self.best_individual = None
        self.best_fitness = float('inf')
        self.nfe = 0

    def initialize_population(self):
        return np.random.uniform(self.function.x_lower, self.function.x_upper,
                               (self.population_size, self.function.dimension))

    def evaluate_fitness(self, individual):
        fitness = self.function(individual)
        self.nfe += 1
        return fitness

    def best_1_mutation(self, target_vector):
        r1, r2 = np.random.choice(np.arange(self.population_size), size=2, replace=False)
        best_vector = self.population[np.argmin([self.evaluate_fitness(ind) for ind in self.population])]
        return best_vector + self.F * (self.population[r1] - self.population[r2])

    def exponential_crossover(self, target_vector, mutant_vector):
        n = len(target_vector)
        j = np.random.randint(0, n)
        trial_vector = target_vector.copy()
        L = 0
        while np.random.rand() < self.CR and L < n:
            trial_vector[j] = mutant_vector[j]
            j = (j + 1) % n
            L += 1
        return trial_vector

    def selection(self, target_vector, trial_vector):
        if self.evaluate_fitness(trial_vector) < self.evaluate_fitness(target_vector):
            return trial_vector
        return target_vector

    def evolve(self):
        while self.nfe < self.max_nfe:
            for i in range(self.population_size):
                # Mutație best/1
                mutant_vector = self.best_1_mutation(self.population[i])

                # Încrucișare exponențială
                trial_vector = self.exponential_crossover(self.population[i], mutant_vector)

                # Selecție
                self.population[i] = self.selection(self.population[i], trial_vector)

                # Actualizare cel mai bun individ
                current_fitness = self.evaluate_fitness(self.population[i])
                if current_fitness < self.best_fitness:
                    self.best_fitness = current_fitness
                    self.best_individual = self.population[i].copy()

            print(f"NFE: {self.nfe}, Best Fitness: {self.best_fitness}")

        return self.best_individual, self.best_fitness

In [62]:
# Definirea funcției de test (exemplu - funcția Sphere)
def sphere_function(x):
    return np.sum(x**2)

# Parametrii algoritmului
population_size = 50
F = 0.8
CR = 0.9
max_nfe = 1000

# Dimensiunea problemei
dimension = 10

# Crearea funcției de test
function = Function(sphere_function, dimension, x_lower=-5.12, x_upper=5.12)

# Crearea algoritmului DE/best/1/exp
de = DE_best_1_exp(function, population_size, F, CR, max_nfe)

# Executarea algoritmului
best_individual, best_fitness = de.evolve()

# Afișarea rezultatelor
print(f"Cel mai bun individ: {best_individual}")
print(f"Fitness-ul cel mai bun: {best_fitness}")

NFE: 2650, Best Fitness: 21.781739615464243
Cel mai bun individ: [ 8.28403042  7.47317645 -1.83406263 -1.12137608  0.20255071  7.32588993
  0.7104588   0.70335452  7.84901545  4.56281478]
Fitness-ul cel mai bun: 21.781739615464243


# **Lab 6**

This lab performs meta-heuristic benchmarking across multiple optimization algorithms:

Problem Suite: Tests algorithms on Sphere, Rastrigin, and Rosenbrock functions in 5D and 10D.

Performance Metrics: Tracks minimum, maximum, mean, and standard deviation of fitness over 10 independent runs.

Convergence Analysis: Generates iteration-vs-fitness plots to visualize algorithm behavior.

Statistical Ranking: Uses non-parametric tests (e.g., Friedman test) to rank algorithms objectively.
The framework leverages GPU acceleration (CuPy) for faster evaluations and emphasizes reproducibility through structured data collection.

In [63]:
import cupy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os

# Create output directory for plots
if not os.path.exists('results'):
    os.makedirs('results')


# Define benchmark functions optimized for GPU
def sphere_function_gpu(x):
    return cp.sum(x ** 2)


def rastrigin_function_gpu(x):
    A = 10
    return A * len(x) + cp.sum(x ** 2 - A * cp.cos(2 * cp.pi * x))


def rosenbrock_function_gpu(x):
    return cp.sum(100 * (x[1:] - x[:-1] ** 2) ** 2 + (1 - x[:-1]) ** 2)


class Function:
    def __init__(self, func, dimension, x_lower, x_upper):
        self.func = func
        self.dimension = dimension
        self.x_lower = x_lower
        self.x_upper = x_upper


class GPUOptimizationAlgorithm:
    def __init__(self, function, population_size):
        self.function = function
        self.population_size = population_size
        self.device = cp.cuda.Device()

    def initialize_population(self):
        return cp.random.uniform(
            self.function.x_lower,
            self.function.x_upper,
            (self.population_size, self.function.dimension)
        )

    def evaluate_population(self, population):
        with self.device:
            return cp.array([self.function.func(ind) for ind in population])


class PopulationV3_SelfAdaptive_GPU(GPUOptimizationAlgorithm):
    def __init__(self, function, population_size=50, alpha=0.5, alpha_change_rate=0.01, max_iterations=1000):
        super().__init__(function, population_size)
        self.alpha = alpha
        self.alpha_change_rate = alpha_change_rate
        self.max_iterations = max_iterations
        self.fitness_history = []  # Add this to track convergence

    def evolve(self):
        population = self.initialize_population()
        fitness = self.evaluate_population(population)
        best_fitness = float(cp.min(fitness))
        self.fitness_history.append(best_fitness)

        for _ in range(self.max_iterations):
            with self.device:
                new_population = cp.array(population)
                mutation = cp.random.normal(0, 1, population.shape)
                new_population += self.alpha * mutation

                new_fitness = self.evaluate_population(new_population)

                improvements = new_fitness < fitness
                population[improvements] = new_population[improvements]
                fitness[improvements] = new_fitness[improvements]

                if cp.sum(improvements) > self.population_size / 5:
                    self.alpha *= (1 + self.alpha_change_rate)
                else:
                    self.alpha *= (1 - self.alpha_change_rate)

                current_best = float(cp.min(fitness))
                self.fitness_history.append(current_best)

        best_idx = cp.argmin(fitness)
        return population[best_idx].get(), float(fitness[best_idx].get())


def run_experiment(config):
    name, func, dim, algo_name, algo_class, run = config
    print(f"Running {algo_name} on {name} (dim={dim}, run={run + 1})")

    function = Function(func, dimension=dim, x_lower=-5.12, x_upper=5.12)
    algorithm = algo_class(function, population_size=50, alpha=0.5,
                           alpha_change_rate=0.01, max_iterations=1000)

    best_individual, best_fitness = algorithm.evolve()

    # Plot convergence for this run
    plt.figure(figsize=(10, 6))
    plt.plot(algorithm.fitness_history)
    plt.title(f"Convergence: {name} - {algo_name} (Dim={dim}, Run={run + 1})")
    plt.xlabel("Iteration")
    plt.ylabel("Best Fitness")
    plt.yscale('log')  # Using log scale for better visualization
    plt.savefig(f'results/convergence_{name}_{dim}d_run{run + 1}.png')
    plt.close()

    return {
        "Function": name,
        "Dimension": dim,
        "Algorithm": algo_name,
        "Run": run + 1,
        "Best Fitness": best_fitness
    }


def main():
    print("Starting optimization experiments...")

    benchmark_functions = {
        "Sphere": sphere_function_gpu,
        "Rastrigin": rastrigin_function_gpu,
        "Rosenbrock": rosenbrock_function_gpu
    }

    dimensions = [5, 10]
    runs_per_problem = 10
    metaheuristics = {
        "PopulationV3_SelfAdaptive_GPU": PopulationV3_SelfAdaptive_GPU,
    }

    configs = [
        (name, func, dim, algo_name, algo_class, run)
        for name, func in benchmark_functions.items()
        for dim in dimensions
        for algo_name, algo_class in metaheuristics.items()
        for run in range(runs_per_problem)
    ]

    start_time = time.time()
    results = []

    # Run experiments sequentially with progress updates
    for config in configs:
        result = run_experiment(config)
        results.append(result)

    results_df = pd.DataFrame(results)

    # Statistical analysis
    summary = results_df.groupby(["Function", "Dimension", "Algorithm"]).agg({
        "Best Fitness": ["min", "max", "mean", "std"]
    }).round(6)

    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

    # Save results
    results_df.to_csv("results/detailed_results.csv", index=False)
    summary.to_csv("results/summary_results.csv")

    print("\nResults have been saved to the 'results' directory:")
    print("- Convergence plots: results/convergence_*.png")
    print("- Detailed results: results/detailed_results.csv")
    print("- Summary results: results/summary_results.csv")


if __name__ == "__main__":
    main()

Starting optimization experiments...
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=5, run=1)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=5, run=2)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=5, run=3)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=5, run=4)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=5, run=5)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=5, run=6)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=5, run=7)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=5, run=8)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=5, run=9)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=5, run=10)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=10, run=1)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=10, run=2)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=10, run=3)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=10, run=4)
Running PopulationV3_SelfAdaptive_GPU on Sphere (dim=10, run=5)
Running Popu